In [1]:
try:
    import micropip
    await micropip.install("pydantic")
    await micropip.install("graphviz")
except Exception:
    pass

In [2]:
import pydantic
from typing import Union

class BaseModel(pydantic.BaseModel):
    class Config:
        extra="forbid"

# Arbres binaires

Un arbre binaire est un arbre dans lequel chaque nœud a au plus deux enfants. On les appellera "left" et "right" (i.e. enfaut "gauche" et enfant "droit").

In [3]:
class BTree(BaseModel):
    left: Union["BTree", None] = None
    right: Union["BTree", None] = None
    value: str


    def _repr_aux(self) -> tuple[int, list[str]]:
        lines = []
        if self.left:
            left_root_index, lines_left = self.left._repr_aux()
            for i, line in enumerate(lines_left):
                if i < left_root_index:
                    lines.append("   " + line)
                elif i == left_root_index:
                    lines.append("╭─╴" + line)
                else:
                    lines.append("│  " + line)
            
        root_index = len(lines)
        lines.append(self.value)
        
        if self.right:
            right_root_index, lines_right = self.right._repr_aux()
            for i, line in enumerate(lines_right):
                if i < right_root_index:
                    lines.append("│  " + line)
                elif i == right_root_index:
                    lines.append("╰─╴" + line)
                else:
                    lines.append("   " + line)
    
        return root_index, lines

    def __repr__(self):
        root_index, lines = self._repr_aux()
        lines_res = []
        for i, line in enumerate(lines):
            if i == root_index:
                lines_res.append("──" + line)
            else:
                lines_res.append("  " + line)
        return "\n".join(lines_res)
BTree.update_forward_refs()

In [49]:
btree_1 = BTree(value="abc")
btree_1

──abc

In [50]:
btree_2 = BTree(left=BTree(value="a"), value="abc")
btree_2

  ╭─╴a
──abc

In [51]:
btree_3 = BTree(left=BTree(value="l"), right=BTree(left=BTree(value="rl"), value="r"), value="abc")
btree_3

  ╭─╴l
──abc
  │  ╭─╴rl
  ╰─╴r

## Recherche dans un arbre

In [94]:
def search_btree(abr: BTree, value: str) ->  bool:
    """
    Renvoie True si un nœud de l'arbre contient value, False sinon.    """

    if abr.value == value:
        return True
    if abr.left:
        found_left = search_btree(abr.left, value)
        if found_left:
            return True
    if abr.right:
        found_right = search_btree(abr.right, value)
        if found_right:
            return True
    print(f"Nœud {abr.value}: pas trouvé {value}")
    return False

In [95]:
search_btree(btree_1, "abc")

True

In [96]:
search_btree(btree_1, "cba")

Nœud abc: pas trouvé cba


False

In [97]:
search_btree(btree_2, "a")

True

In [98]:
search_btree(btree_2, "b")

Nœud a: pas trouvé b
Nœud abc: pas trouvé b


False

In [99]:
search_btree(btree_3, "rl")

Nœud l: pas trouvé rl


True

# Arbres binaires de recherche

Un arbre binaire de recherche est un arbre binaire, qui a la propriété suivante: pour chaque nœud $n$, la valeur de tous les nœuds à gauche est plus petite que la valeur de $n$, et la valeur de tous les nœuds à droite est plus grande que la valeur de $n$.

Les arbres binaires de recherche sont utilisés pour créer par exemple des indexes, ou des ensembles. Nous allons ici implémeter une classe `Set` (ensemblee en anglais).

## Exemples

Parmi les arbres suivants, lesquels sont des arbres binaires de recherche ?

In [7]:
ex1 = BTree(
    left=BTree(value="cca"), 
    value="ccc", 
    right=BTree(value="ccd")
)
ex1

  ╭─╴cca
──ccc
  ╰─╴ccd

In [8]:
ex2 = BTree(
    left=BTree(value="cda"), 
    value="ccc", 
    right=BTree(value="ccd")
)
ex2

  ╭─╴cda
──ccc
  ╰─╴ccd

In [9]:
ex3 = BTree(
    left=BTree(
        left=BTree(
            value="aaa",
        ),
        value="aca",
        right=BTree(
            value="acd",
            right=BTree(value="ace")
        )
    ),
    value="cca", 
    right=BTree(
        left=BTree(value="ccb"),
        value="ccd"
    )
)
ex3

     ╭─╴aaa
  ╭─╴aca
  │  ╰─╴acd
  │     ╰─╴ace
──cca
  │  ╭─╴ccb
  ╰─╴ccd

In [10]:
ex4 = BTree(
    left=BTree(
        left=BTree(
            value="aaa",
        ),
        value="aca",
        right=BTree(
            value="acd",
            right=BTree(value="ace")
        )
    ),
    value="cca", 
    right=BTree(
        left=BTree(value="ccb"),
        value="ccd",
        right=BTree(value="aab")
    )
)
ex4

     ╭─╴aaa
  ╭─╴aca
  │  ╰─╴acd
  │     ╰─╴ace
──cca
  │  ╭─╴ccb
  ╰─╴ccd
     ╰─╴aab

## Teste de propriété

On va implémenter la fonction `is_abr` qui permet de tester si un arbre est un abr

In [19]:
def max_btree(btree: BTree) -> str:
    """retourne la plus grande valeur de l'arbre btree (qui n'est pas forcément un abr"""
    res = btree.value
    if btree.left:
        max_left = max_btree(btree.left)
        if max_left > res:
            res = max_left
    if btree.right:
        max_right = max_btree(btree.right)
        if max_right > res:
            res = max_right
    return res

In [20]:
def min_btree(btree: BTree) -> str:
    """retourne la plus petite valeur de l'arbre btree (qui n'est pas forcément un abr"""
    res = btree.value
    if btree.left:
        min_left = min_btree(btree.left)
        if min_left < res:
            res = min_left
    if btree.right:
        min_right = min_btree(btree.right)
        if min_right < res:
            res = min_right
    return res

In [41]:
def is_abr(abr: BTree) -> bool:
    """Retourne True si abr est un arbre binaire de recherche (False sinon).
    
    On devra vérifier: 
        - que le max de l'arbre à gauche est plus petit que la valeur du nœud
        - que l'arbre à gauche est un abr
        - que le min de l'arbre à droite est plus grand que la valeur du nœud
        - que l'arbre à droite est un abr
    """
    if abr.left:
        max_left = max_btree(abr.left)
        if max_left > abr.value:
            print(f"Max left {max_left}, value {abr.value}")
            return False
        if not is_abr(abr.left):
            print(f"not abr ?\n{abr.left}")
            return False
    if abr.right:
        min_right = min_btree(abr.right)
        if min_right < abr.value:
            print(f"Min right {min_right}, value {abr.value}")
            return False
        if not is_abr(abr.right):
            print(f"not abr ?\n{abr.right}")
            return False
    return True

In [42]:
is_abr(ex1)

True

In [43]:
is_abr(ex2)

Max left cda, value ccc


False

In [44]:
is_abr(ex3)

True

In [45]:
is_abr(ex4)

Min right aab, value cca


False

## Recherche dans un arbre binaire de recherche

In [ ]:
def search_abr(abr: BTree, value: str) ->  bool:
    """
    Renvoie True si un nœud de l'arbre contient value, False sinon.
    On suppose que l'arbre vérifie la propritété des arbres binaires de recherche
    """

    if abr.value == value:
        return True
    elif value < abr.value and abr.left:
        return search_abr(abr.left)
    elif value > abr.value and abr.right:
        return search_abr(abr.right) 